Library

In [ ]:
from utils.utils_dta_processing import *
from utils.utils_ml_train import *

Data

In [ ]:
df_roe = pd.read_csv('../../data/data_for_modelling/df_roe.csv')
roe_input, roe_test = input_test_split(df_roe)